# Calcul flux données internet

On a, par jour, un flux moyen entrant et un flux moyen sortant. Je ne sais pas à partir de quoi cette moyenne est obtenue (1 échantillonage d'une heure, une moyenne sur toute la journée, etc.)

In [10]:
import pandas as pd
import math

traffic_df = pd.read_csv("traffic-donnees-sete.csv")
traffic_df

,Hostname,Target,Sample Date YMD,Sample Time HHMM,Count in seconds,Incoming traffic in bps (avg day),Outgoing traffic in bps (avg day),Peak inbound traffic in bps,Peak outbound traffic in bps
0,sites,GigabitEthernet1/0,11/23/2020,1:00,1606089600,55192.0,117222.6,601766.4,2346130.7
1,sites,GigabitEthernet1/0,11/24/2020,1:00,1606176000,5351484.2,1745443.2,46186475.3,21301345.1
2,sites,GigabitEthernet1/0,11/25/2020,1:00,1606262400,7087875.1,1063973.7,77554496.2,14717147.1
3,sites,GigabitEthernet1/0,11/26/2020,1:00,1606348800,5238180.8,2139920.4,72566815.4,32529461.6
4,sites,GigabitEthernet1/0,11/27/2020,1:00,1606435200,6030543.9,5191834.1,35059495.4,70644815.9
...,...,...,...,...,...,...,...,...,...
360,sites,GigabitEthernet1/0,11/18/2021,1:00,1637193600,28440854.6,9821430.1,175365040.4,159933589.3
361,sites,GigabitEthernet1/0,11/19/2021,1:00,1637280000,27491672.0,6812135.1,175481504.2,88729496.2
362,sites,GigabitEthernet1/0,11/20/2021,1:00,1637366400,22971880.9,7349417.9,175626957.8,37371580.7
363,sites,GigabitEthernet1/0,11/21/2021,1:00,1637452800,311307.2,1581631.7,6464377.3,100715441.7


On passe du flux de données (en bps) à une quantité de données, sur l'ensemble de la journée.
12h ou 24h ?

## Flux sortant

In [23]:
n_hours_in_day = 12
traffic_df['data_out_Gb'] = (3600 * n_hours_in_day * traffic_df['Outgoing traffic in bps (avg day)'] / 10**9)

In [24]:
traffic_df['data_out_Gb'].mean()

218.28438043397264

## Flux entrant

In [25]:
traffic_df['data_in_Gb'] = (3600 * n_hours_in_day * traffic_df['Incoming traffic in bps (avg day)'] / 10**9)

In [26]:
traffic_df['data_in_Gb'].mean()

331.36398191737004

# Données manquantes

Il nous faut les données de présence sur site, pour déduire une consommation individuelle, qui permettra de calculer le bilan de 2019.
Il nous faut aussi savoir exactement comment est calculé le flux moyen (pendant présence sur site, ou la nuit est comptée aussi)